In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
os.chdir('/Users/jed/Documents/JED/Dataiku/drive-download-20250106T224222Z-001')
!ls

census_income_additional_info.pdf census_income_metadata.txt
census_income_learn.csv           census_income_test.csv


In [3]:
#load the train and test datasets into pandas df
train_data=pd.read_csv('census_income_learn.csv', header=None)
test_data=pd.read_csv('census_income_test.csv', header=None)


In [4]:
if len(train_data.columns) == len(test_data.columns):
    print('Both train and test datasets have same no. of columns:', len(test_data.columns))

# There are no column/feature names, will need to add these from metadata.txt file
print('n= ', len(train_data), ' for the training dataset') 
print('n= ', len(test_data), ' for the testing dataset') 

Both train and test datasets have same no. of columns: 42
n=  199523  for the training dataset
n=  99762  for the testing dataset


In [5]:
# need to add column names from metadata.txt file
column_names=['AAGE',
 'ACLSWKR',
 'ADTIND',
 'ADTOCC',
 'AGI',
 'AHGA',
 'AHRSPAY',
 'AHSCOL',
 'AMARITL',
 'AMJIND',
 'AMJOCC',
 'ARACE',
 'AREORGN',
 'ASEX',
 'AUNMEM',
 'AUNTYPE',
 'AWKSTAT',
 'CAPGAIN',
 'CAPLOSS',
 'DIVVAL',
 'FEDTAX',
 'FILESTAT',
 'GRINREG',
 'GRINST',
 'HHDFMX',
 'HHDREL',
 'MARSUPWT',
 'MIGMTR1',
 'MIGMTR2',
 'MIGMTR4',
 'MIGSAME',
 'MIGSUN',
 'NOEMP',
 'PARENT',
 'PEARNVAL',
 'PEFNTVTY',
 'PEMNTVTY',
 'PENATVTY',
 'PRCITSHP',
 'PTOTVAL',
 'SEOTR',
 'TAXINC',
 'VETQVA',
 'VETYN',
 'WKSWORK']

# AGI column is adjusted gross income, which should be removed as it is the target feature
# so should FEDTAX, PERNVAL, PTOTVAL all earning related columns 
# year of survey completion column name is missing from the txt file but is in the df
cols_to_remove=['AGI', 'FEDTAX','PEARNVAL','PTOTVAL','TAXINC', ]

for col in cols_to_remove:
    if col in column_names:
        column_names.remove(col)

column_names = column_names + ['YEAROFSUR']

# # assigned TARGET as the last column
column_names=column_names+['TARGET']

train_data.columns=column_names
test_data.columns=column_names

# # make the target variable binary as this is a binary classification task
train_data['TARGET_bin']=np.where(train_data.TARGET==' 50000+.',1,0)
test_data['TARGET_bin']=np.where(test_data.TARGET==' 50000+.',1,0)

# drop the original target variable
train_data.drop('TARGET', axis=1, inplace=True)
test_data.drop('TARGET', axis=1, inplace=True)
if len(column_names)== len(train_data.columns) == len(test_data.columns):
    print('Column names have been added to the train and test datasets')

Column names have been added to the train and test datasets
